# FarOpt Basic Example Notebook

### Uninstall any existing versions and reinstall

In [1]:
!pip uninstall faropt -y

Found existing installation: faropt 0.1.5
Uninstalling faropt-0.1.5:
  Successfully uninstalled faropt-0.1.5


In [2]:
!pip install -e .

Obtaining file:///home/ec2-user/SageMaker/faropt/sdk
  Running setup.py develop for faropt
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Import and initialize a FarOpt object

In [3]:
from faropt import FarOpt

In [4]:
fo = FarOpt()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:root:FarOpt backend is ready!


In [5]:
fo.list_jobs()

jobid:2020-11-12-01-21-13-953576ea-455c-4afe-ba6f-09ff616b0a9a | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-12-01-21-13-953576ea-455c-4afe-ba6f-09ff616b0a9a/source.zip
jobid:2020-11-12-00-26-29-d25426f3-4615-4b5f-9f40-4d5c7dad0c4c | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-12-00-26-29-d25426f3-4615-4b5f-9f40-4d5c7dad0c4c/source.zip
jobid:2020-11-12-01-44-16-0aa1a401-e2e9-4b28-846d-7d23e2291fc2 | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-12-01-44-16-0aa1a401-e2e9-4b28-846d-7d23e2291fc2/source.zip
jobid:2020-11-12-01-22-07-f40e8b89-e2e7-42d7-82b8-964d300a11ed | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-12-01-22-07-f40e8b89-e2e7-42d7-82b8-964d300a11ed/source.zip
jobid:2020-11-12-01-10-52-a14f2df2-031f-4424-9b0f-ed0ad71d1155 | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-12-01-10-52-a14f2df2-031f-4424-9b0f-ed0ad71d1155/source.zip
jobid:2020-11-12-01-24-15-3caf57a2-aed7-4cf4-8cb2-22ccccf54410 | bucket:faropt-s

In [7]:
fo.run_s3_job(bucket='faropt-s3bucketfbfa637e-jauzfgds5cug',key='2020-11-12-01-25-18-8f565343-5960-4406-b453-b8a5213b6436/source.zip')

INFO:root:Downloading source...
INFO:root:Configured job!
INFO:root:Submitting job
INFO:root:Submitted job! id: 2020-11-12-02-15-00-cd7c6fc0-8676-497d-b81b-052481bfbc1b


In [ ]:
fo.wait()

PROVISIONING


# Sample Vehicle routing problem run using OR tools

In [ ]:
!pygmentize ../tests/src/main.py

### Configure and submit the job

In [ ]:
fo.configure('../tests/srcga/')

In [ ]:
fo.submit()

### Wait for job to complete

look for INFO:root:JOB COMPLETED!

In [ ]:
fo.wait()

In [ ]:
fo.list_recipes()

In [ ]:
fo.logs()

### Optionally Add this problem as a standard recipe

In [ ]:
fo.add_recipe('cvrp_problem')

### Rerun this recipe at any time

In [ ]:
fo.run_recipe('cvrp_problem')

### List past jobs and recipes

In [ ]:
fo.list_jobs()

In [ ]:
fo.list_recipes()

In [ ]:
fo.configure('../tests/srcga/')

In [ ]:
fo.add_recipe('ga_problem')

In [ ]:
!pygmentize ../tests/srcga/main.py